In [14]:
# ! pip install transformers

In [2]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

bucket = sess.default_bucket()
prefix = "sagemaker/pytorch-bert-sentiment-analysis"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
sentiment_analysis.save_pretrained("./model")

In [5]:
!ls -rtlh ./model/

total 257M
-rw-rw-r-- 1 ec2-user ec2-user  774 Apr  7 07:34 config.json
-rw-rw-r-- 1 ec2-user ec2-user 256M Apr  7 07:34 model.safetensors
-rw-rw-r-- 1 ec2-user ec2-user 1.3K Apr  7 07:34 tokenizer_config.json
-rw-rw-r-- 1 ec2-user ec2-user  125 Apr  7 07:34 special_tokens_map.json
-rw-rw-r-- 1 ec2-user ec2-user 227K Apr  7 07:34 vocab.txt
-rw-rw-r-- 1 ec2-user ec2-user 695K Apr  7 07:34 tokenizer.json


In [6]:
!cd model && tar czvf ../model.tar.gz *

config.json
model.safetensors
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.txt


In [7]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)
print(os.path.join(bucket, key))

sagemaker-eu-west-2-661082688832/sagemaker/pytorch-bert-sentiment-analysis/model.tar.gz


In [8]:
pretrained_model_data = "s3://{}/{}".format(bucket, key)
pretrained_model_data

's3://sagemaker-eu-west-2-661082688832/sagemaker/pytorch-bert-sentiment-analysis/model.tar.gz'

In [12]:
!pygmentize Code/inference.py

import json
from transformers import pipeline

JSON_CONTENT_TYPE = 'application/json'


def model_fn(model_dir):
    sentiment_analysis = pipeline(
        "sentiment-analysis",
        model=model_dir,
        tokenizer=model_dir,
        return_all_scores=True
    )

    return sentiment_analysis


def input_fn(serialized_input_data, content_type=JSON_CONTENT_TYPE):
    if content_type == JSON_CONTENT_TYPE:
        input_data = json.loads(serialized_input_data)
        return input_data

    else:
        raise Exception('Requested unsupported ContentType in Accept: ' + content_type)
        return


def predict_fn(input_data, model):
    print('Got input Data: {}'.format(input_data))
    return model(input_data)


def output_fn(prediction_output, accept=JSON_CONTENT_TYPE):
    if accept == JSON_CONTENT_TYPE:
        return json.dumps(prediction_output), accept

    raise Exception('Requested unsupported ContentType in Accept: ' + accept)


In [22]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=pretrained_model_data,
    role=role,
    framework_version="2.1.0",
    source_dir="Code",
    py_version="py310",
    entry_point="inference.py",
)

In [23]:
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type="ml.m5.large")

------!

In [24]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [25]:
result = predictor.predict("Never allow the same bug to bite you twice.")
result

[[{'label': 'NEGATIVE', 'score': 0.817720890045166},
  {'label': 'POSITIVE', 'score': 0.1822790503501892}]]

In [26]:
result = predictor.predict(
    [
        "Never allow the same bug to bite you twice.",
        "The best part of Amazon SageMaker is that it makes machine learning easy.",
    ]
)
result

[[{'label': 'NEGATIVE', 'score': 0.817720890045166},
  {'label': 'POSITIVE', 'score': 0.1822790503501892}],
 [{'label': 'NEGATIVE', 'score': 0.002658776007592678},
  {'label': 'POSITIVE', 'score': 0.9973412156105042}]]

In [27]:
predictor.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
